In [1]:
from utils.hole_tracker import HoleTracker, StructuredDeque
import numpy as np

Tracker = HoleTracker(
    10, 10, 10,
    
    logging_level   = "DEBUG",
    tiebreak_method = "KDE-N4-BW0.1",
    update_method   = "AVG-N5",

    thr_ddetect= 0.1,
    thr_imugaps= 0.5,
    thr_inframe= 2.0,
    thr_offrame= 5.0,
    
    imu_hist_minlen = 4,
    imu_hist_maxlen = 20,
    )

print(Tracker)
print("something else")



[DEBUG]: initialized tracker!

╔════════════════════════════════════════ TRACKER SUMMARY ═════════════════════════════════════════╗

[FREQ_VISIBILITY_CHECK ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ 10 Hz]
[FREQ_VISIBILITY_CHECK ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ 10 Hz]
[FREQ_VISIBILITY_CHECK ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ 10 Hz]
[LOGGING_LEVEL ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ DEBUG]
[TIEBREAK_METHOD ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ KDE]
[TIEBREAK_N ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ 4]
[TIEBREAK_BW ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ 0.1]
[UPDATE_METHOD ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ AVG]
[UPDATE_N ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄

In [2]:
from utils.hole_tracker import HoleTracker, StructuredDeque
import numpy as np

aaa = StructuredDeque(10, datatype=[
    ("asdf", np.float32, (1, )), 
    ("ghc", np.float32, (4, )),
])
aaa.append((1.0, [2, 3, 4, 5]))
aaa.append((1.0, list(np.random.rand(4, ))))
aaa.append((1.0, list(np.random.rand(4, ))))
aaa.append((1.0, list(np.random.rand(4, ))))
aaa.append((1.0, list(np.random.rand(4, ))))
aaa.append((1.0, list(np.random.rand(4, ))))


print(aaa)

+----------------------+------------------+------------------+------------------+------------------+
│         asdf         │      ghc[0]      │      ghc[1]      │      ghc[2]      │      ghc[3]      │
+----------------------+------------------+------------------+------------------+------------------+
│       1.000000       │     2.000000     │     3.000000     │     4.000000     │     5.000000     │
│       1.000000       │     0.329093     │     0.937608     │     0.109258     │     0.224036     │
│       1.000000       │     0.792421     │     0.199758     │     0.653837     │     0.667744     │
│       1.000000       │     0.496670     │     0.409082     │     0.704205     │     0.647675     │
│       1.000000       │     0.196779     │     0.462071     │     0.997217     │     0.699857     │
│       1.000000       │     0.695410     │     0.421166     │     0.750853     │     0.277173     │
+----------------------+------------------+------------------+------------------+----------

In [3]:
from utils.hole_tracker_V3 import HoleTracker
import numpy as np

Tracker = HoleTracker(
    10, 10, 10,
    tiebreak_method = "KDE-N4-BW0.1",
    update_method   = "AVG-N5",

    imu_hist_minlen = 4,
    imu_hist_maxlen = 20,
    
    thr_ddetect= 10.0

    )

Tracker.do_new_imu_logic( 0, np.array([[1, 0, 0]+[0]*3]))
Tracker.do_new_imu_logic( 1, np.array([[0, 1, 0]+[0]*3]))
Tracker.do_new_imu_logic( 2, np.array([[0, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 3, np.array([[0, 1, 0]+[0]*3]))
Tracker.do_new_imu_logic( 4, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(1.3, np.random.rand(5, 3))
Tracker.do_new_detection_logic(1.9, np.random.rand(5, 3))
Tracker.do_new_detection_logic(2.8, np.random.rand(5, 3))
Tracker.do_new_detection_logic(3.4, np.random.rand(5, 3))

# print(Tracker._flag_new_detection)
Tracker.do_new_imu_logic( 5, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 6, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(5.6, np.random.rand(5, 3))

Tracker.do_new_imu_logic( 7, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 8, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(7.46, np.random.rand(5, 3))

Tracker.do_new_imu_logic( 9, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic(10, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(8.99, np.random.rand(5, 3))

print("\n")
print("imu_data", Tracker._imu_data)
print("estimate", Tracker._p_estimate)
print("detections",Tracker._p_detection)



ModuleNotFoundError: No module named 'utils.hole_tracker_V3'

In [ ]:
import numpy as np

aaa = np.random.rand(3, )
bbb = np.random.rand(3, )

display(np.concatenate([aaa, bbb]))

array([0.39899971, 0.3148621 , 0.22787628, 0.01518808, 0.23634809,
       0.5139903 ])